In [11]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from time import time
from tqdm import tqdm

import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt

import tensorflow as tf
from pix2vid2 import make_model
from utils import check_tf_gpu, describe_data, load_data

msteps = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
NUM_REALIZATIONS = 1000
NX,  NY,  NZ = 64, 64, 1
NTT, NT1, NT2 = 40, 20, len(msteps)
HIDDEN = [16, 64, 256]

folder     = 'SlopingAquiferSmall/simulationsVE'
sec2year   = 365.25 * 24 * 60 * 60
Darcy      = 9.869233e-13
psi2pascal = 6894.76
co2_rho    = 686.5266
milli      = 1e-3
mega       = 1e6
check_tf_gpu()

--------------------------------------------------------------
------------------------ VERSION INFO ------------------------
TF version: 2.15.0 | Keras: 3.5.0 | # Device(s) available: 2
TF Built with CUDA? True | CUDA: 12.2 | cuDNN: 8
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
--------------------------------------------------------------



In [13]:
gt = sio.loadmat('{}/grids/Gt.mat'.format(folder), simplify_cells=True)['Gt']
gtops = gt['cells']['z'].reshape(NX,NY,order='F')
gvols = gt['parent']['cells']['volumes'].reshape(NX,NY,order='F')
ghght = gt['cells']['H'].reshape(NX,NY,order='F')

deltatime = sio.loadmat('{}/data/time_arr.mat'.format(folder), simplify_cells=True)['time_arr']
timesteps = np.cumsum(deltatime)
timesteps_inj = timesteps[:20]
timesteps_mon = timesteps[20:][msteps]
#timesteps_mon = timesteps[[19, 24, 29, 34, 39]]

print('timesteps: {} | deltatime: {}'.format(len(timesteps), np.unique(deltatime)))
print('injection: {}'.format(timesteps_inj))
print('monitoring: {}'.format(timesteps_mon))

timesteps: 40 | deltatime: [  0.5 100. ]
injection: [ 0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5  7.
  7.5  8.   8.5  9.   9.5 10. ]
monitoring: [ 110.  310.  510.  710.  910. 1110. 1310. 1510. 1710. 1910.]


In [15]:
datas, norms = load_data(folder)
X_data, c_data, y1_data, y2_data = datas.values()

X: (1000, 64, 64, 5) | c: (1000, 20, 5)
y1: (1000, 20, 64, 64, 2) | y2: (1000, 20, 64, 64, 1)
porosity     - min: -0.62 | max: 2.36
logperm      - min: -2.20 | max: 2.49
wells        - min: 0.00 | max: 1.00
tops         - min: 0.00 | max: 1.00
volumes      - min: 0.00 | max: 1.00
controls     - min: 0.00 | max: 0.97
pressure_1   - min: 0.00 | max: 15.39
saturation_1 - min: 0.00 | max: 1.00
saturation_2 - min: 0.00 | max: 1.00


In [16]:
train_idx = np.load('{}/models/training_idx.npy'.format(folder))
test_idx  = np.setdiff1d(range(len(X_data)), train_idx)

tridx = train_idx[:20]
teidx = test_idx[:20]

X_train  = tf.cast(X_data[tridx], tf.float32)
c_train  = tf.cast(c_data[tridx], tf.float32)
y1_train = tf.cast(y1_data[tridx], tf.float32)
y2_train = tf.cast(y2_data[tridx], tf.float32)

X_test  = tf.cast(X_data[teidx], tf.float32)
c_test  = tf.cast(c_data[teidx], tf.float32)
y1_test = tf.cast(y1_data[teidx], tf.float32)
y2_test = tf.cast(y2_data[teidx], tf.float32)

In [ ]:
model_inj, model_mon = make_model(hidden=HIDDEN, nt1=NT1, nt2=NT2)

model_inj.load_weights('models/pix2vid-v2-inj.weights.h5')
model_mon.load_weights('models/pix2vid-v2-mon.weights.h5')

y1_train_pred = model_inj.predict([X_train, c_train], verbose=False)
y2_train_pred = model_mon.predict([X_train, c_train], verbose=False)

y1_test_pred = model_inj.predict([X_test, c_test], verbose=False)
y2_test_pred = model_mon.predict([X_test, c_test], verbose=False)

y1_train_pred = np.array(y1_train_pred)
y2_train_pred = np.array(y2_train_pred)
y1_test_pred = np.array(y1_test_pred)
y2_test_pred = np.array(y2_test_pred)

X_train = np.array(X_train)
c_train = np.array(c_train)
y1_train = np.array(y1_train)
y2_train = np.array(y2_train)

X_test = np.array(X_test)
c_test = np.array(c_test)
y1_test = np.array(y1_test)
y2_test = np.array(y2_test)

print('Train - y1: {} | y2: {}'.format(y1_train_pred.shape, y2_train_pred.shape))
print('Test - y1: {} | y2: {}'.format(y1_test_pred.shape, y2_test_pred.shape))

In [ ]:
sample = 13

fig, axs = plt.subplots(3, 10, figsize=(15,5), sharex=True, sharey=True)
for j in range(10):
    ax1, ax2, ax3 = axs[0,j], axs[1,j], axs[2,j]
    k = j*2
    
    t = y1_train[sample, k, ..., 0]
    p = y1_train_pred[sample, k, ..., 0]
    e = np.linalg.norm(t.reshape(1,64,64)-p.reshape(1,64,64), axis=0)

    im1 = ax1.imshow(t, cmap='jet')
    im2 = ax2.imshow(p, cmap='jet')
    im3 = ax3.imshow(e, cmap='binary')
    [plt.colorbar(ii, pad=0.04, fraction=0.046) for ii in [im1,im2,im3]]
    ax1.set_title('t={}'.format(timesteps[k]))
    
plt.tight_layout()
plt.show()

fig, axs = plt.subplots(3, 10, figsize=(15,5), sharex=True, sharey=True)
for j in range(10):
    ax1, ax2, ax3 = axs[0,j], axs[1,j], axs[2,j]
    k = j*2
    
    t = y1_train[sample, k, ..., 1]
    p = y1_train_pred[sample, k, ..., 1]
    e = np.linalg.norm(t.reshape(1,64,64)-p.reshape(1,64,64), axis=0)

    im1 = ax1.imshow(t, cmap='jet')
    im2 = ax2.imshow(p, cmap='jet')
    im3 = ax3.imshow(e, cmap='binary')
    [plt.colorbar(ii, pad=0.04, fraction=0.046) for ii in [im1,im2,im3]]
    ax1.set_title('t={}'.format(timesteps[k]))
    
plt.tight_layout()
plt.show()

fig, axs = plt.subplots(3, 8, figsize=(15,5), sharex=True, sharey=True)
for j in range(8):
    ax1, ax2, ax3 = axs[0,j], axs[1,j], axs[2,j]

    t = y2_train[sample, j, ..., -1]
    p = y2_train_pred[sample, j, ..., -1]
    e = np.linalg.norm(t.reshape(1,64,64)-p.reshape(1,64,64), axis=0)

    im1 = ax1.imshow(t, cmap='jet')
    im2 = ax2.imshow(p, cmap='jet')
    im3 = ax3.imshow(e, cmap='binary')
    [plt.colorbar(ii, pad=0.04, fraction=0.046) for ii in [im1,im2,im3]]
    ax1.set_title('t={}'.format(timesteps_mon[j]))

plt.tight_layout()
plt.show()

In [ ]:
hues = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple']
plt.figure(figsize=(15,7.5))
for i in range(15):
    plt.subplot(3, 5, i+1)
    d = c_data[i]
    nw = len(np.unique(d.sum(0)))-1
    for w in range(nw):
        for t in range(len(timesteps_inj)-1):
            plt.hlines(d[t,w], timesteps_inj[t], timesteps_inj[t+1], color=hues[w])
            plt.vlines(timesteps_inj[t+1], d[t,w], d[t+1,w], color=hues[w], ls=':', lw=0.5)
    plt.title('R{} | ({})'.format(i, nw))
    plt.xlim(0.5,10)
    plt.ylim(0,None)
    plt.xticks(np.arange(1,11))
    plt.grid(True, which='both', alpha=0.25)
    plt.xlabel('Time (years)') if i > 9 else None
    plt.ylabel('Inj. Rate (MT CO2)') if i % 5 == 0 else None
plt.tight_layout()
plt.show()

***
### Optimization

In [ ]:
porevol = np.load('{}/simulations/data/porevol.npy'.format(folder))
porevol.shape

In [ ]:
sample = 123

dc = c_train[sample].sum()
print(dc)

dd = (y2_train[sample, -1].squeeze() * porevol[sample]) / 1e3 / mega * co2_rho
print(np.sum(dd))

plt.imshow(dd, cmap='jet')
plt.colorbar()
plt.show()